In [1]:
%env THEANO_FLAGS=floatX=float32,device=gpu0,lib.cnmem=0.5

env: THEANO_FLAGS=floatX=float32,device=gpu0,lib.cnmem=0.5


In [2]:
%matplotlib inline
import os
import h5py
import json
import pandas as pd
import scipy as scp
import cv2
import numpy as np 
from keras.preprocessing import image
from keras.optimizers import SGD
# from deeplearningmodels.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from shutil import copyfile
import shutil
from convnetfactory import ConvNetFactory
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.applications.vgg16 import VGG16
from convnetfactory import ConvNetFactory, VGG_16
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential

Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5105)


In [3]:
def minority_balance_dataframe_by_multiple_categorical_variables(df, var=['label']):
    minority_class_combination_count = df.groupby(var).apply(lambda x: x.shape[0]).min()
    df = df.groupby(var).apply(lambda x: x.sample(minority_class_combination_count)).drop(var, axis=1).reset_index().set_index('level_1')
    df.sort_index(inplace=True)
    return df

In [4]:
#prepare data

EXCLUDE_LABELS = ['top','other','noclass']

with open('labels.json', 'r') as labels:
    labels = json.loads(labels.read())

df_labels = pd.DataFrame(data={'label': list(labels.values()), 'file_name': list(labels.keys())})
df_labels = df_labels[~df_labels.label.isin(EXCLUDE_LABELS)]
# df_labels = df_labels[df_labels.label.isin(['front-side','back-side'])]


df_labels.set_value(df_labels.index, 'is_train',True)

# df_labels = minority_balance_dataframe_by_multiple_categorical_variables(df_labels)
# 

train, test = train_test_split(df_labels, test_size = 0.2, stratify=df_labels.label)
df_labels.set_value(train.index, 'is_train', True)
df_labels.set_value(test.index, 'is_train', False)
df_labels = df_labels.reindex_axis(sorted(df_labels.columns), axis=1)
df_labels.label.value_counts()


# train.label.value_counts()

front-side    5246
back-side     1084
side           952
front          512
back           196
Name: label, dtype: int64

In [5]:
# create flow-from-directory-structure
# cwd = os.getcwd()
BASE_PATH = '/home/i008/googledrive/Projects/detect-good-car-picture/'
EXP_NAME = 'exp'
FULL_EXP_PATH = os.path.join(BASE_PATH, EXP_NAME)
TRAIN_PATH = os.path.join(FULL_EXP_PATH, 'train')
TEST_PATH = os.path.join(FULL_EXP_PATH, 'test')

if os.path.exists(FULL_EXP_PATH):
    shutil.rmtree(FULL_EXP_PATH)
    
os.makedirs(os.path.join(FULL_EXP_PATH))
os.makedirs(os.path.join(FULL_EXP_PATH, 'train'))
os.makedirs(os.path.join(FULL_EXP_PATH, 'test'))

for label in df_labels.label.unique():
    os.makedirs(os.path.join(FULL_EXP_PATH, 'train', label))
    os.makedirs(os.path.join(FULL_EXP_PATH, 'test', label))       
        
for _, row in df_labels.iterrows():
    path, is_train, label = row[0], row[1], row[2]
    file_name = os.path.split(path)[-1]
    if is_train:
        copy_to = os.path.join(FULL_EXP_PATH, 'train', label, file_name)
        copyfile(path, copy_to)
    else:
        copy_to = os.path.join(FULL_EXP_PATH, 'test', label, file_name)
        copyfile(path, copy_to)

In [8]:
CHANNELS = 3
target_size = (150, 150)
imgRows = target_size[0]
imgCols = target_size[1]
N_CLASSES = len(df_labels.label.unique())
c = (CHANNELS, 150, 150, 2)

def create_model(img_rows, img_cols, n_classes):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(3, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))

    model.add(Dense(32))
    model.add(Activation('relu'))

    model.add(Dropout(0.3))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    
    return model

model = create_model(*target_size, n_classes=N_CLASSES)

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [1]:
# imd_train = image.ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         vertical_flip=True

# )

# imd_test = image.ImageDataGenerator(rescale=1./255)

# imd_train_flow = imd_train.flow_from_directory(TRAIN_PATH,
#                                                color_mode='rgb',
#                                                target_size=target_size,
#                                                batch_size=32,
#                                                class_mode='categorical'
#                                               )
# imd_test_flow = imd_test.flow_from_directory(TEST_PATH,
#                                              color_mode='rgb',
#                                              batch_size=32,
#                                              class_mode='categorical',
#                                              target_size=target_size)

# training_history = model.fit_generator(
#     imd_train_flow, 
#     validation_data=imd_test_flow, 
#     samples_per_epoch=df_labels[df_labels.is_train].shape[0], 
#     nb_epoch=30, 
#     verbose=True,
#     nb_val_samples=200,
#     class_weight={0: 1 , 1: 0.1808, 2: 0.38, 3: 0.03, 4: 0.2}
# )



# from keras import backend as K
# IMAGE_ORDERING = 'tf'
# IMAGENET_TARGET_SHAPE = (224, 224)
# K.set_image_dim_ordering(IMAGE_ORDERING)
# # images = [np.expand_dims(image.img_to_array(image.load_img(p, target_size=None)), axis=0) for p in df.file_name[:20]]

In [25]:
# imd_test_flow = imd_test.flow_from_directory(TEST_PATH, 
#                                              batch_size=300,
#                                              class_mode='binary',
#                                              target_size=target_size)


from sklearn.preprocessing import LabelEncoder


# # im = image.load_img(df_labels[df_labels.is_train].iloc[0]['file_name'],target_size=(300, 300))
# # I = np.expand_dims(image.img_to_array(im), 0)

t = []
l = []
for f  in df_labels[df_labels.is_train == False].file_name:
    t.append(image.img_to_array(image.load_img(f, target_size=(150, 150))) * 1./255)
    
labs = df_labels[df_labels.is_train == False].label.tolist()    

M = np.concatenate([np.expand_dims(z, axis=0) for z in t], axis=0)
M.shape


preds = model.predict_classes(M)


ims = []
for f  in df_labels[df_labels.is_train == False].iloc[np.where((real == preds.reshape(1, -1)[0]) == False)].file_name:
    ims.append(image.load_img(f))
    
sum(real == preds.reshape(1, -1)[0]) / (len(real) * 1.)


# ims[15]

1598/1598 [==============================] - 0s     


0.76470588235294112

In [33]:
le = LabelEncoder()
le.fit(labs)
real = le.transform(labs)
real

pred_y  = le.inverse_transform(preds)
real_y = le.inverse_transform(real)
pred_y
real_y
real


label_encoder = LabelEncoder().fit(labs)
label_encoder.inverse_transform(preds)
# from pandas_ml import ConfusionMatrix
# cm =  ConfusionMatrix(real_y, pred_y)
# cm.plot(normalized=True, backend='seaborn', max_colors=30)
# cm.classification_report

array([3, 3, 3, ..., 3, 1, 4])

In [12]:
dict(zip(le.transform(le.classes_), le.classes_))

{0: 1 , 1: 0.1808, 2: 0.38, 3: 0.03, 4: 0.2}

{0: 1, 1: 0.1808, 2: 0.38, 3: 0.03, 4: 0.2}

In [241]:
# h5file.create_dataset('images_tf', (len(df),224,224,3), maxshape=(None,224,224,3), chunks=True, dtype=float)
# h5file.create_dataset('labels', shape=(len(df),), dtype="S10")

# images_ds = h5file.get('images_tf')
# labels_ds = h5file.get('labels')


# model.predict(HDF5Matrix('cars4.h5','images_tf',start=0, end=10))

dict(196/df_labels.label.value_counts())

{u'back': 1.0,
 u'back-side': 0.18081180811808117,
 u'front': 0.3828125,
 u'front-side': 0.037361799466260011,
 u'side': 0.20588235294117646}

In [6]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.vgg19 import VGG19

In [5]:
# resnet = ResNet50(include_top=False)
# vgg19 = VGG19()


def load_image_keras(image_path, gray=False, target_size=(300, 300)):
        im = image.load_img(image_path, grayscale=gray, target_size=target_size)
        imarray = image.img_to_array(im)
        imarray = np.expand_dims(imarray, axis=0)
        imarray = preprocess_input(imarray)
        return imarray

In [166]:
# im = load_image_keras(df_labels.file_name.iloc[0], target_size=(224, 224))


ims = [load_image_keras(p, target_size=(224, 224)) for p in df_labels.file_name]
imsconc = np.concatenate(ims, axis=0)

# load_image_keras(df_labels.file_name.iloc[0])

In [167]:
preds = resnet.predict(imsconc)
# preds = vgg19.predict(imsconcs)

In [171]:
X = preds.flatten().reshape(preds.shape[0], -1)
X.shape

(1832, 2048)

In [131]:
# model = Sequential()
# model.add(Dense(2048, activation='relu', input_dim=2048))
# # model.add(Dense(3024, activation='relu'))
# # model.add(Dense(2024, activation='relu'))
# # model.add(Dense(1024, activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [132]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [172]:
y = LabelEncoder().fit_transform(df_labels.label)

In [141]:
y

array([1, 0, 0, ..., 0, 0, 1])

In [2]:
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'],
)



model.fit(X, y, validation_split=0.2, nb_epoch=75, verbose=1)

# preds[0].flatten()

In [123]:
from sklearn.ensemble import RandomForestClassifier

In [153]:
rfc = RandomForestClassifier(oob_score=True, n_estimators=500, n_jobs=3)
rfc.fit(X, df_labels.label)
rfc.oob_score_

0.90489130434782605

In [68]:
preds_resnet_full =resnet.predict(imsconc)

In [63]:
from keras.applications.imagenet_utils import decode_predictions

In [3]:
# decode_predictions(preds_resnet_full)

In [5]:
# resnet_top = ResNet50(include_top=True)
# decode_predictions(resnet_top.predict(imsconc[10:15]))

In [128]:
df_labels.label.value_counts()

front-side    736
back-side     736
Name: label, dtype: int64

In [8]:
# model = Sequential()

# model.add(Dense(100, input_dim=2048, init="uniform", activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(30, init="uniform", activation="relu"))
# model.add(Dense(20, init="uniform", activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(10, init="uniform", activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(1, init="uniform", activation="sigmoid"))
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# model.fit(X, y, validation_split=0.33, nb_epoch=100, batch_size=10)

In [8]:
from predict import label_encoder, model
from utils import load_image_keras

In [9]:
# label_encoder.classes_

In [10]:
back_side = '/home/i008/cars_train/00433.jpg'
front_image = '/home/i008/cars_train/00433.jpg'

im  = load_image_keras(front_image, target_size=(150, 150))
im=im*1/255
predictions = model.predict_classes(im)
print(label_encoder.inverse_transform(predictions))